In [126]:
# import
import pandas as pd
import numpy as np
import selenium
import sqlalchemy
from urllib import parse
import requests
from bs4 import BeautifulSoup as BS
import os
import json
from tqdm import tqdm
import pickle
from geopy.geocoders import Nominatim
import subprocess
import time

# custom python lib
import src.naver_geocoding_api as nga
import src.evchargestations as evcs
import src.connect_to_db as cdb


In [174]:
with open("./data/3fec80b505ceeecf.csv", "r", encoding='cp949') as f:
    ev_counts = pd.read_csv(f)


In [177]:
ev_counts.drop(columns=['합계'], axis=1, inplace=True)

In [180]:
col_list = ev_counts.columns.tolist()

In [184]:
ev_counts_melt = ev_counts.melt(['연도'], col_list).copy()


In [187]:
with open("./data/ev_counts_melt.csv", "w", encoding='utf-8') as f:
    ev_counts_melt.rename(columns={'variable':'지역', 'value':'대수'}).to_csv(f)

In [188]:
with open("./results/rest_areas.json", "r", encoding='utf-8') as f:
    rest_areas = json.load(f)

In [189]:
rest_areas_df = pd.DataFrame(rest_areas['list'])

In [191]:
with open("./results/rest_areas_df.pkl", "wb") as f:
    pickle.dump(rest_areas_df, f)